In [1]:
import pandas as pd
import database_connect as db_connect
import stock_marker_binance
import numpy as np

In [2]:
CONFIG = db_connect.load_config()

In [3]:
def hours_to_milliseconds(hours):
    return hours * 60 * 60 * 1000

In [4]:
def insert_data_to_db(conn, df):
    with conn.cursor() as cursor:
        for row in df.itertuples():
            query = f"INSERT INTO trading_bot_historydataeth (timestamp, open, high, low, close, volume, \
            close_time, quote_asset_volume, num_trades) \
            VALUES ('{row.timestamp}', {row.open}, {row.high}, {row.low}, {row.close}, {row.volume}, \
            {row.close_time}, {row.quote_asset_volume}, {row.num_trades})"
            cursor.execute(query)
            conn.commit()

### Adding historical data to the database.

In [ ]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [25]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,01/01/2020 00:00,129.16,129.19,128.68,128.87,7769.17336,1.577840e+12,1.000930e+06,2504
1,01/01/2020 01:00,128.87,130.65,128.78,130.64,11344.65516,1.577840e+12,1.474278e+06,4885
2,01/01/2020 02:00,130.63,130.98,130.35,130.85,7603.35623,1.577850e+12,9.940256e+05,3046
3,01/01/2020 03:00,130.85,130.89,129.94,130.20,4968.55433,1.577850e+12,6.473610e+05,2818
4,01/01/2020 04:00,130.21,130.74,130.15,130.20,3397.90747,1.577850e+12,4.430067e+05,2264


In [27]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [28]:
for row in df.itertuples():
    print(tuple(row))
    break

(0, '01/01/2020 00:00', 129.16, 129.19, 128.68, 128.87, 7769.17336, 1577840000000.0, 1000929.742, 2504)


In [29]:
# insert_data_to_db(conn, df) # Commented to avoid inserting data to db

In [32]:
conn.close()

### Update database with fresh data from Binance

In [4]:
# Every hour the script updates the database with the latest data from the API.
# The script is run using a cron job.

# Identifying the last timestamp in the database and using that as the starting point for the API call.

In [36]:
# Identifying the last timestamp in the database and using that as the starting point for the API call.
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [37]:
with conn.cursor() as cursor:
    cursor.execute("SELECT MAX(timestamp) FROM trading_bot_historydataeth")
    result = cursor.fetchone()
    print(result)
conn.close()

(datetime.datetime(2024, 2, 7, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=18000))),)


In [38]:
INTERVAL_START = result[0]

print(INTERVAL_START)

INTERVAL_START = INTERVAL_START.replace(tzinfo=None)
print(INTERVAL_START -  stock_marker_binance.UNIX_TIME_START)
INTERVAL_START = int((INTERVAL_START -  stock_marker_binance.UNIX_TIME_START).total_seconds()) * 1000 + 1
print(INTERVAL_START)




2024-02-07 06:00:00+05:00
19760 days, 6:00:00
1707285600001


In [39]:
url = 'https://api3.binance.com/api/v3/time'

df_time_stop = pd.read_json(url, typ='series')

TIME_STOP = df_time_stop.iloc[0]
print(TIME_STOP)

# TIME_STOP = int ( time.mktime(TIME_STOP.timetuple())*1000)
TIME_STOP = int(TIME_STOP.timestamp()) * 1000
print(TIME_STOP)

2024-02-07 10:12:03.452000
1707300723000


In [41]:
SYMBOL = 'ETHUSDT'
INTERVAL = '1h'
df_total = pd.DataFrame()

In [42]:
while INTERVAL_START < TIME_STOP:
      df=stock_marker_binance.import_stock_data(SYMBOL, INTERVAL, startTime =INTERVAL_START,  limit=984)
      df.rename(
          columns={0:'timestamp', 1:'open', 2:'high', 3:'low', 4:'close',5:'volume', 6:'close_time', 7:'quote_asset_volume',
                8:'num_trades', 9:'taker_buy_base_asset_volume',10:'taker_buy_quote_asset_volume', 11:'unused_field'},
         inplace=True)
      df.drop(columns=['taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'unused_field'], inplace=True)
      if len(df_total)>0:
            df_total = pd.concat([df_total, df], ignore_index=True)
      else:
            df_total = df.copy()
      INTERVAL_START = INTERVAL_START + hours_to_milliseconds(984)
      print(len(df_total))

4


In [43]:
df_total.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2024-02-07 07:00:00,2364.09,2368.41,2360.44,2362.92,6186.9306,1707292799999,1.462528e+07,18315
1,2024-02-07 08:00:00,2362.91,2365.45,2356.94,2363.25,10160.5082,1707296399999,2.398462e+07,21284
2,2024-02-07 09:00:00,2363.26,2377.42,2361.42,2371.61,9173.1951,1707299999999,2.173872e+07,26255
3,2024-02-07 10:00:00,2371.60,2372.11,2368.51,2370.07,932.9221,1707303599999,2.211184e+06,4130


In [44]:
conn = db_connect.connect(CONFIG)
insert_data_to_db(conn, df_total)
conn.close()

Connected to the PostgreSQL server.


### Recalculate the additional columns

In [106]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [107]:
with conn.cursor() as cursor:
    cursor.execute("SELECT id, timestamp, open,	high,	low,	close,	volume,	close_time, \
                    quote_asset_volume,	num_trades \
                FROM trading_bot_historydataeth \
                ORDER BY timestamp DESC \
                LIMIT 1084")
    result = cursor.fetchall()
conn.close()

In [108]:
df = pd.DataFrame(result, columns=['id', 'timestamp', 'open', 'high', 'low', 
                                   'close', 'volume', 'close_time', 'quote_asset_volume', 'num_trades'])

In [109]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year

In [110]:
df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])
print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])
print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 
                                                #  'quote_asset_volume', 'num_trades'
                                                 ])
print("Complete adding RSI")


df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI


In [111]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RATIO_close_and_STD192,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close
0,39193,0.500000,-8.660254e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,53.072935,52.313467,27.586207,64.074278,69.197699,73.345154,60.974797,57.433627,52.130657,50.200471
1,39192,0.707107,-7.071068e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,52.289809,52.381125,100.000000,59.950810,69.425792,75.539632,63.418157,59.704071,52.730219,50.314028
2,39191,0.866025,-5.000000e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,51.503319,52.547921,100.000000,82.926829,71.003452,70.679721,65.078982,60.475730,53.265658,50.843922
3,39190,0.965926,-2.588190e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,50.372328,52.373532,100.000000,80.537535,81.084648,68.485240,65.842872,61.637226,53.524448,51.018082
4,39189,1.000000,6.123234e-17,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,49.147178,52.130302,100.000000,100.000000,75.639365,66.862416,66.580463,61.379636,53.974138,51.130688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,37811,0.258819,-9.659258e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.246572,42.031553,48.341625,48.776978,71.060657,54.936954,57.581115,42.731113,49.504484,49.851127
696,37810,0.500000,-8.660254e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.233467,42.026535,0.000000,34.702381,76.449143,55.824956,58.133078,42.867880,49.572050,49.942764
697,37809,0.707107,-7.071068e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.384260,42.157878,63.660477,55.994843,70.822042,60.580755,62.779054,44.068106,49.873482,50.067255
698,37808,0.866025,-5.000000e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.324732,42.100210,67.796610,57.451499,65.070729,59.618976,60.009311,44.327585,50.181674,50.021291


In [112]:
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [113]:
df_result

,id,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,...,RATIO_close_and_STD192,RATIO_close_and_STD384,RSI_3_close,RSI_6_close,RSI_12_close,RSI_24_close,RSI_48_close,RSI_96_close,RSI_192_close,RSI_384_close
0,39193,0.500000,-8.660254e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,53.072935,52.313467,27.586207,64.074278,69.197699,73.345154,60.974797,57.433627,52.130657,50.200471
1,39192,0.707107,-7.071068e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,52.289809,52.381125,100.000000,59.950810,69.425792,75.539632,63.418157,59.704071,52.730219,50.314028
2,39191,0.866025,-5.000000e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,51.503319,52.547921,100.000000,82.926829,71.003452,70.679721,65.078982,60.475730,53.265658,50.843922
3,39190,0.965926,-2.588190e-01,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,50.372328,52.373532,100.000000,80.537535,81.084648,68.485240,65.842872,61.637226,53.524448,51.018082
4,39189,1.000000,6.123234e-17,-0.994522,-0.104528,5.000000e-01,0.866025,0.000000,1.00000,2024,...,49.147178,52.130302,100.000000,100.000000,75.639365,66.862416,66.580463,61.379636,53.974138,51.130688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,37811,0.258819,-9.659258e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.246572,42.031553,48.341625,48.776978,71.060657,54.936954,57.581115,42.731113,49.504484,49.851127
696,37810,0.500000,-8.660254e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.233467,42.026535,0.000000,34.702381,76.449143,55.824956,58.133078,42.867880,49.572050,49.942764
697,37809,0.707107,-7.071068e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.384260,42.157878,63.660477,55.994843,70.822042,60.580755,62.779054,44.068106,49.873482,50.067255
698,37808,0.866025,-5.000000e-01,-0.951057,0.309017,-2.449294e-16,1.000000,-0.781831,0.62349,2023,...,48.324732,42.100210,67.796610,57.451499,65.070729,59.618976,60.009311,44.327585,50.181674,50.021291


In [69]:
cols = list(df_result.columns)

In [70]:
cols.remove('id')
cols

['hour_sin',
 'hour_cos',
 'day_sin',
 'day_cos',
 'mon_sin',
 'mon_cos',
 'weekday_sin',
 'weekday_cos',
 'year',
 'RATIO_close_and_MA3',
 'RATIO_close_and_MA6',
 'RATIO_close_and_MA12',
 'RATIO_close_and_MA24',
 'RATIO_close_and_MA48',
 'RATIO_close_and_MA96',
 'RATIO_close_and_MA192',
 'RATIO_close_and_MA384',
 'RATIO_quote_asset_volume_and_MA3',
 'RATIO_quote_asset_volume_and_MA6',
 'RATIO_quote_asset_volume_and_MA12',
 'RATIO_quote_asset_volume_and_MA24',
 'RATIO_quote_asset_volume_and_MA48',
 'RATIO_quote_asset_volume_and_MA96',
 'RATIO_quote_asset_volume_and_MA192',
 'RATIO_quote_asset_volume_and_MA384',
 'RATIO_num_trades_and_MA3',
 'RATIO_num_trades_and_MA6',
 'RATIO_num_trades_and_MA12',
 'RATIO_num_trades_and_MA24',
 'RATIO_num_trades_and_MA48',
 'RATIO_num_trades_and_MA96',
 'RATIO_num_trades_and_MA192',
 'RATIO_num_trades_and_MA384',
 'RATIO_close_and_STD3',
 'RATIO_close_and_STD6',
 'RATIO_close_and_STD12',
 'RATIO_close_and_STD24',
 'RATIO_close_and_STD48',
 'RATIO_close

In [77]:
conn = db_connect.connect(CONFIG)

Connected to the PostgreSQL server.


In [85]:
for index, row in df_result.iterrows():
    update_sql = "UPDATE trading_bot_historydataeth SET "
    for col in cols:
        update_sql += f"{col} = {row[col]}, "
    update_sql = update_sql[:-2]
    update_sql += f" WHERE id = {int(row['id'])} and hour_sin IS NULL"
    print(update_sql)
    with conn.cursor() as cursor:
        cursor.execute(update_sql)
        conn.commit()
conn.close()

UPDATE trading_bot_historydataeth SET hour_sin = 0.49999999999999994, hour_cos = -0.8660254037844387, day_sin = -0.9945218953682733, day_cos = -0.10452846326765423, mon_sin = 0.49999999999999994, mon_cos = 0.8660254037844387, weekday_sin = 0.0, weekday_cos = 1.0, year = 2024.0, RATIO_close_and_MA3 = 1.000561621503521, RATIO_close_and_MA6 = 1.0007024758948513, RATIO_close_and_MA12 = 1.0070094783539936, RATIO_close_and_MA24 = 1.0165235500740883, RATIO_close_and_MA48 = 1.044229435801771, RATIO_close_and_MA96 = 1.058276996491584, RATIO_close_and_MA192 = 1.0524464865979952, RATIO_close_and_MA384 = 1.0405984191056041, RATIO_quote_asset_volume_and_MA3 = 1.571583971140065, RATIO_quote_asset_volume_and_MA6 = 1.5447807483143643, RATIO_quote_asset_volume_and_MA12 = 1.5987890229528219, RATIO_quote_asset_volume_and_MA24 = 1.6812292778278446, RATIO_quote_asset_volume_and_MA48 = 1.7334150398193697, RATIO_quote_asset_volume_and_MA96 = 2.2532709056600986, RATIO_quote_asset_volume_and_MA192 = 2.90655358

InFailedSqlTransaction: ОШИБКА:  текущая транзакция прервана, команды до конца блока транзакции игнорируются
